In [1]:
# IMPORTING PACKAGES
# ------------------------------------------------------
import pandas as pd
import numpy as np
import os
import re
import warnings

# VISUALIZATION
import seaborn as sns
%matplotlib inline
import matplotlib.pyplot as plt
%matplotlib inline
pd.options.display.max_columns = 0
pd.options.display.max_colwidth =10

# CONFIGURATIONS
# ------------------------------------------------------
warnings.filterwarnings("ignore")

In [2]:
os.chdir("data")

In [3]:
df = pd.read_csv("CleanedData.csv")

In [4]:
df["Category_name"].value_counts()

tv, television, led                2266
drive, storage, hard               2105
headphone, earbud, bluetooth       1801
tv, 4k, television                 1636
headphone                          1580
receiver, amplifier, home          1541
car, gps, dash                     1464
camera, shoot, point               1386
tablet, computer, ipad             1363
camera, mirrorless                 1302
laptop, computer                   1210
speaker, portable, bluetooth       1202
player, ipod                       1098
sound, speaker, home               1088
car, speaker, subwoofer            1072
speaker, audio                     1035
case, computer                      926
laptop, computer, macbook           925
speaker, bluetooth, mobile          841
phone, cell                         789
mount, tv, stand                    783
camera, camcorder, action           750
wireless, speaker, audio            696
lens, camera                        683
monitor, computer                   654


Model for "Speakers"

In [5]:
dfx = df[df["Category_name"].str.contains("speaker",case=False)]
dfx= dfx[dfx["Year"]==2017]

In [6]:
dfx = dfx.reset_index(drop=True)

In [7]:
dfx.sort_values(by=['Date'],ignore_index=True)

,id,Name,Brand,Description,Category_name,Merchant,Actual_price,Discount_price,isSale,Discount_per,Imp_count,Day,Month,Year,Week,Month_Name,Day_Name,Date
0,AVphKa...,SRS-ZR...,Sony,Stereo...,wirele...,bhphot...,198.00,198.00,No,0.0,2,28,2,2017,9,February,Tuesday,2017-0...
1,AVpgcA...,Sony -...,Sony,Stereo...,speake...,bhphot...,238.00,238.00,No,0.0,2,28,2,2017,9,February,Tuesday,2017-0...
2,AVphS0...,Sony -...,Sony,Audio ...,speake...,bhphot...,348.00,348.00,No,0.0,3,1,3,2017,9,March,Wednesday,2017-0...
3,AVpfHc...,SP-FS5...,Pioneer,Speake...,speake...,Bestbu...,129.99,129.99,No,0.0,13,2,3,2017,9,March,Thursday,2017-0...
4,AVphe2...,Refere...,Klipsch,Stereo...,speake...,Bestbu...,549.98,549.98,No,0.0,19,2,3,2017,9,March,Thursday,2017-0...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6156,AVpgoU...,JBL Cl...,JBL,Stereo...,speake...,kohls.com,59.99,59.99,No,0.0,46,30,12,2017,52,December,Saturday,2017-1...
6157,AVpgoU...,JBL Cl...,JBL,Stereo...,speake...,kohls.com,59.99,59.99,No,0.0,46,31,12,2017,52,December,Sunday,2017-1...
6158,AVpgoU...,JBL Cl...,JBL,Stereo...,speake...,kohls.com,59.99,59.99,No,0.0,46,31,12,2017,52,December,Sunday,2017-1...
6159,AVpgoU...,JBL Cl...,JBL,Stereo...,speake...,kohls.com,59.99,59.99,No,0.0,46,31,12,2017,52,December,Sunday,2017-1...


In [8]:
#Grouping data by Name and Week number.
dfx_mod = dfx.groupby(['Name', 'Week']).agg({'Discount_price':'mean','Imp_count':'sum'}).reset_index()


In [9]:
dfx_mod.sample(5)

,Name,Week,Discount_price,Imp_count
676,Detail...,27,76.99,8
2828,Sony -...,20,237.49,22
198,Alpine...,22,176.99,4
441,Cerwin...,41,198.99,3
444,Cerwin...,9,129.99,14


In [10]:

#New X dataframe.
x_new = dfx_mod.pivot(index='Week', columns='Name' ,values='Discount_price')
print("Structure of DataFrame = ",x_new.shape)
x_new.sample(5)

Structure of DataFrame =  (42, 223)


Name,151 SE Outdoor Environmental Speakers (White),"2) New Kicker 41KSC6934 6x9 3-Way 300 Watt Car Audio Coaxial Speakers KSC693""",251 Outdoor Environmental Speakers (White),"AW-650 All-Weather Outdoor Speaker (Pair, White)","AW6500 All-Weather Outdoor Speaker (White, Single)",Acoustimass 6 Series V Home Theater Speaker System (Black),"Alpine - 5 x 7"" 2-Way Coaxial Car Speakers with Poly-Mica Cones (Pair) - Black""","Alpine - 6 x 9"" 2-Way Coaxial Car Speakers with Polypropylene Cones (Pair) - Black""","Alpine - 6-1/2 2-Way Component Car Speakers with Poly-Mica Cones (Pair) - Black""",Alpine Electronics PWE-S8-WRA Powered Subwoofer for 2011-Up Jeep Wrangler,Alpine KTP-445U Universal Power Pack Amplifier for Use w/ Aftermarket Head Units,Alpine PDXM12 1200W Mono RMS Digital Amplifier,"Alpine SPR-60 6-1/2 Coaxial 2-Way Type-R Speaker Set""","Atrend-Bbox 10 Dual Front-Fire Enclosure for Dodge Ram 19962002""","Atrend-Bbox E12DSV B Box Series 12 Dual Vented Enclosure with Shared Chamber""","BIC America - 6-1/2 Floor Speaker (Each) - Black""",BOOM 2 Wireless Bluetooth Speaker - Indigo,BRAVEN BRV-HD Wireless Bluetooth Speaker [28 Hour Playtime][Water Resistant] Built-In 8800 mAh Power Bank Charger - Black,"Bic America Rtr1530 15 Rtr Series 3-way Tower Speaker""",Boombox+ Bluetooth Wireless Speaker,Bose SoundLink Color Bluetooth Speaker (Black),Bose® - Soundlink® Color Bluetooth Speaker II - Coral Red,Bose® - Virtually Invisible® 591 In-Ceiling Speakers (Pair) - White,Bose® - Virtually Invisible® 791 In-Ceiling Speakers II (Pair) - White,Boytone - 2500W 2.1-Ch. Home Theater System - Black Diamond,Boytone - Portable Bluetooth Speaker - Blue,Braven BRV-BLADE Wireless Portable Bluetooth Speaker [22 Hour Playtime][Waterproof] 4000 mAh Power Bank Charger - Black,Braven Balance Portable Bluetooth Wireless Speaker,Cerwin Vega - SL Series 2-Way Center-Channel Loudspeaker - Black,Cerwin Vega - SL Series 2-Way Center-Channel Speaker - Black,Chant Mini Portable Bluetooth Wireless Speaker (Navy),Chant Sport BT Portable Audio System (Black),Chant Sport BT Portable Audio System (Natural),"Definitive Technology - DT Series 6.5 2-Way In-Ceiling Speaker (Each) - Black""","Definitive Technology - DT Series 8 2-Way In-Ceiling Speaker (Each) - Black""","Definitive Technology - Disappearing Dual 6-1/2 In-Wall Speaker (Each) - Black""",Definitive Technology - Mythos 2-way 175 W Speaker - Pack of 1 - Black,Definitive Technology - ProCinema 600 5.1-Channel Home Theater Speaker System - Black,"Definitive Technology - SuperCube 2000 7-1/2 650W Powered Subwoofer - Black""","Definitive Technology - W9 Dual 5-1/4 180W 2-Way Wireless Speaker (Each) for Streaming Music - Black""",...,Sony - 5.1-Ch. 3D / Smart Blu-ray Home Theater System - Black,"Sony - Core Series 10 115W Active Subwoofer - Black""","Sony - Core Series Dual 5 3-Way Floorstanding Speaker (Each) - Black""",Sony - High Power XB60 Portable Bluetooth Speaker - Black,Sony - XB20 Portable Bluetooth Speaker - Blue,Sony - XB7 Extra Bass Audio System with Bluetooth - Black,Sony HTST9 Soundbar with Wireless Subwoofer Bluetooth and Google Cast,Sony SRSHG1/BLK Hi-Res Wireless Speaker- Charcoal black,Sony SRSXB3/BLK Portable Wireless Speaker with Bluetooth (Black),Sony Ultra-Portable Bluetooth Speaker,Soundcast - Portable Bluetooth Speaker - Gray/black,"SpeakerCraft - 6-1/2 In-Ceiling Speakers (5-Pack) - White""","SpeakerCraft - Profile CRS6 Two 6-1/2 In-Ceiling Speaker (Each) - Black""",Toshiba - 2.0-Channel Soundbar with 16-Watt Digital Amplifier - Black,Tough Portable Bluetooth Speaker (Black and Silver),Virtually Invisible 891 In-Wall Speakers (Pair),Vizio 38 5.1 Sound Bar System - Black (sb3851-C0),YSP-4300 Digital Sound Projector Wireless Active Subwoofer (Black),YU2 Powered Desktop Speakers (Glossy Red),YU2 Powered Desktop Speakers (Matte Black),YU2 Powered Desktop Speakers (Matte Gray),"Yamaha - 10 250W Powered Subwoofer - High-Gloss Piano Black""","Yamaha - 2.1-Channel Soundbar System with 6-1/2 Wirel

In [14]:
# FOR X VALUES
#Filling missing values with fillna + forward filling
x_new_ = pd.DataFrame(x_new.to_records())
x_new_.fillna(method='ffill', inplace=True)
x_new_.fillna(method='bfill', inplace=True)

In [20]:
# FOR Y VALUES
#Filling missing values with fillna + forward filling
y_new = dfx_mod.pivot(index='Week', columns='Name', values='Imp_count')
y_new_ = pd.DataFrame(y_new.to_records())
y_new_.fillna(method='ffill', inplace=True)
y_new_.fillna(method='bfill', inplace=True)

Price elasticity = Change in Quanity sold / Change in the Price

In [19]:
for col in x_new_.columns[1:]:
    print(col)

151 SE Outdoor Environmental Speakers (White)
2) New Kicker 41KSC6934 6x9 3-Way 300 Watt Car Audio Coaxial Speakers KSC693"
251 Outdoor Environmental Speakers (White)
AW-650 All-Weather Outdoor Speaker (Pair, White)
AW6500 All-Weather Outdoor Speaker (White, Single)
Acoustimass 6 Series V Home Theater Speaker System (Black)
Alpine - 5 x 7" 2-Way Coaxial Car Speakers with Poly-Mica Cones (Pair) - Black"
Alpine - 6 x 9" 2-Way Coaxial Car Speakers with Polypropylene Cones (Pair) - Black"
Alpine - 6-1/2 2-Way Component Car Speakers with Poly-Mica Cones (Pair) - Black"
Alpine Electronics PWE-S8-WRA Powered Subwoofer for 2011-Up Jeep Wrangler
Alpine KTP-445U Universal Power Pack Amplifier for Use w/ Aftermarket Head Units
Alpine PDXM12 1200W Mono RMS Digital Amplifier
Alpine SPR-60 6-1/2 Coaxial 2-Way Type-R Speaker Set"
Atrend-Bbox 10 Dual Front-Fire Enclosure for Dodge Ram 19962002"
Atrend-Bbox E12DSV B Box Series 12 Dual Vented Enclosure with Shared Chamber"
BIC America - 6-1/2 Floor Spea

#### Forward Selection Method

In [41]:
from sklearn.datasets import load_iris
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.linear_model import LogisticRegression

X, y = load_iris(return_X_y=True)
lr = LogisticRegression()
sfs = SequentialFeatureSelector(lr, n_features_to_select=3,direction="forward")

sfs.fit(X, y)

sfs.get_support()
sfs.transform(X).shape
print('Selected features:', sfs.get_support(indices=True))

Selected features: [0 2 3]


#### Backward Selection Method

In [40]:
from sklearn.datasets import load_iris
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.linear_model import LogisticRegression

X, y = load_iris(return_X_y=True)
lr = LogisticRegression()
sfs = SequentialFeatureSelector(lr, n_features_to_select=2,direction="backward")

sfs.fit(X, y)

sfs.get_support()
sfs.transform(X).shape
print('Selected features:', sfs.get_support(indices=True))

Selected features: [2 3]
